In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import flopy
from shapely.geometry import Point
import os
from pathlib import Path

def extract_modflow_usg_heads(model_ws, model_name, shapefile_path, output_csv=None, layer=1):
    """
    Extract hydraulic head data from MODFLOW-USG model at specified locations.
    
    Parameters:
    -----------
    model_ws : str
        Path to the model workspace directory
    model_name : str
        Name of the MODFLOW model (without extension)
    shapefile_path : str
        Path to shapefile containing point locations for head extraction
    output_csv : str, optional
        Output CSV file path. If None, saves as '{model_name}_heads.csv'
    layer : int, optional
        Layer number to extract heads from (1-based indexing)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing coordinates and head values for each location
    """
    
    # Load the MODFLOW-USG model
    print(f"Loading MODFLOW-USG model from {model_ws}")
    try:
        # Try to load existing model
        ml = flopy.modflow.Modflow.load(f"{model_name}.nam", model_ws=model_ws)
    except:
        print("Error loading model. Make sure the model files exist in the specified directory.")
        return None
    
    # Load the shapefile with monitoring locations
    print(f"Loading monitoring locations from {shapefile_path}")
    try:
        locations_gdf = gpd.read_file(shapefile_path)
        print(f"Found {len(locations_gdf)} monitoring locations")
    except:
        print("Error loading shapefile. Check the file path and format.")
        return None
    
    # Get model grid information
    if hasattr(ml, 'disu'):  # Unstructured grid
        print("Processing unstructured grid (DISU)")
        grid = ml.disu
        # Get cell centers
        if hasattr(grid, 'get_xcyc'):
            xc, yc = grid.get_xcyc()
        else:
            # Alternative method for getting cell centers
            xc = grid.nodelay[0]  # This might need adjustment based on your flopy version
            yc = grid.nodelay[1]
    else:
        print("Model does not appear to be MODFLOW-USG (no DISU package found)")
        return None
    
    # Load head output file
    head_file = os.path.join(model_ws, f"{model_name}.hds")
    if not os.path.exists(head_file):
        print(f"Head file not found: {head_file}")
        return None
    
    print("Loading head data...")
    try:
        # Load heads using flopy
        hds = flopy.utils.HeadFile(head_file)
        times = hds.get_times()
        print(f"Found {len(times)} time steps")
    except:
        print("Error loading head file")
        return None
    
    # Prepare results DataFrame
    results_list = []
    
    # Process each monitoring location
    for idx, location in locations_gdf.iterrows():
        loc_x = location.geometry.x
        loc_y = location.geometry.y
        
        # Get location ID if available
        loc_id = location.get('ID', location.get('Name', location.get('id', f'Point_{idx}')))
        
        print(f"Processing location {loc_id} at ({loc_x:.2f}, {loc_y:.2f})")
        
        # Find the nearest cell to the monitoring location
        distances = np.sqrt((xc - loc_x)**2 + (yc - loc_y)**2)
        nearest_cell = np.argmin(distances)
        nearest_distance = distances[nearest_cell]
        
        print(f"  Nearest cell: {nearest_cell}, distance: {nearest_distance:.2f}")
        
        # Extract heads for all time steps at this location
        for time_idx, time in enumerate(times):
            try:
                # Get heads for this time step
                heads = hds.get_data(totim=time)
                
                # For unstructured grid, heads are typically 1D array
                if heads.ndim > 1:
                    # If 3D array, select the specified layer
                    head_value = heads[layer-1, nearest_cell] if heads.ndim == 3 else heads[nearest_cell]
                else:
                    head_value = heads[nearest_cell]
                
                # Skip dry cells (usually represented as very negative values)
                if head_value < -999:
                    head_value = np.nan
                
                results_list.append({
                    'Location_ID': loc_id,
                    'X_Coord': loc_x,
                    'Y_Coord': loc_y,
                    'Layer': layer,
                    'Time_Step': time_idx + 1,
                    'Time': time,
                    'Cell_Index': nearest_cell,
                    'Distance_to_Cell': nearest_distance,
                    'Head': head_value
                })
                
            except Exception as e:
                print(f"  Error extracting head for time {time}: {e}")
                continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results_list)
    
    if len(results_df) == 0:
        print("No head data extracted!")
        return None
    
    # Save to CSV
    if output_csv is None:
        output_csv = os.path.join(model_ws, f"{model_name}_heads.csv")
    
    results_df.to_csv(output_csv, index=False)
    print(f"Head data exported to: {output_csv}")
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Total records: {len(results_df)}")
    print(f"Unique locations: {results_df['Location_ID'].nunique()}")
    print(f"Time steps: {results_df['Time_Step'].nunique()}")
    print(f"Head range: {results_df['Head'].min():.2f} to {results_df['Head'].max():.2f}")
    
    return results_df


# Example usage
if __name__ == "__main__":
    
    # Configuration
    MODEL_WORKSPACE = r"...\workspace"  # Update this
    MODEL_NAME = "your_model_name"  # Update this
    SHAPEFILE_PATH = r"...\monitoring_locations.shp"  # Update this 
    OUTPUT_CSV = r"...\extracted_heads.csv"  # Optional, will auto-generate if None
    
    
    # Extract heads
    try:
        head_data = extract_modflow_usg_heads(
            model_ws=MODEL_WORKSPACE,
            model_name=MODEL_NAME,
            shapefile_path=SHAPEFILE_PATH,
            output_csv=OUTPUT_CSV,
            layer=1  # Extract from layer 1
        )
        
        if head_data is not None:
            print("\nFirst few records:")
            print(head_data.head())
            
            # Optional: Create pivot table for easier analysis
            pivot_table = head_data.pivot_table(
                index=['Location_ID', 'X_Coord', 'Y_Coord'], 
                columns='Time_Step', 
                values='Head'
            )
            pivot_output = OUTPUT_CSV.replace('.csv', '_pivot.csv') if OUTPUT_CSV else 'head_data_pivot.csv'
            pivot_table.to_csv(pivot_output)
            print(f"Pivot table saved to: {pivot_output}")
            
    except Exception as e:
        print(f"Error during execution: {e}")
        print("Please check your file paths and model setup.")


def plot_head_time_series(csv_file, location_ids=None, output_plot=None):
    """
    Plot time series of heads for specified locations.
    
    Parameters:
    -----------
    csv_file : str
        Path to the CSV file with head data
    location_ids : list, optional
        List of location IDs to plot. If None, plots all locations.
    output_plot : str, optional
        Path to save the plot. If None, displays interactively.
    """
    
    try:
        import matplotlib.pyplot as plt
        
        df = pd.read_csv(csv_file)
        
        if location_ids is None:
            location_ids = df['Location_ID'].unique()[:10]  # Limit to first 10 if not specified
        
        plt.figure(figsize=(12, 8))
        
        for loc_id in location_ids:
            loc_data = df[df['Location_ID'] == loc_id].sort_values('Time')
            plt.plot(loc_data['Time'], loc_data['Head'], marker='o', label=loc_id, linewidth=2)
        
        plt.xlabel('Time')
        plt.ylabel('Hydraulic Head')
        plt.title('Hydraulic Head Time Series')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        if output_plot:
            plt.savefig(output_plot, dpi=300, bbox_inches='tight')
            print(f"Plot saved to: {output_plot}")
        else:
            plt.show()
            
    except ImportError:
        print("Matplotlib not available for plotting. Install with: pip install matplotlib")
    except Exception as e:
        print(f"Error creating plot: {e}")

